<a href="https://colab.research.google.com/github/manjitullal/foursquare/blob/master/LSTM_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from google.colab import drive
import warnings
warnings.filterwarnings('ignore')

In [ ]:
drive.mount('/content/drive')
path = "/content/drive/My Drive/dataset/foursquare"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
%%time
Checkin_columns = ['UserID','VenueID','Year','Month','Date','Hour']
Checkin = pd.read_csv(path+'/Checkin.txt', sep=',', skiprows=1, names=Checkin_columns)

CPU times: user 638 ms, sys: 160 ms, total: 798 ms
Wall time: 2.6 s


In [ ]:
Checkin.head()

,UserID,VenueID,Year,Month,Date,Hour
0,u1302,v47,2012,2,24,11
1,u45,v132,2012,2,24,11
2,u24844,v86,2012,2,24,11
3,u896,v248,2012,2,24,11
4,u5020,v29,2012,2,24,11


In [ ]:
#find the users with most events 

Checkin.groupby(by='UserID')['UserID'].count().sort_values(ascending=False)

UserID
u1205     1303
u2651     1100
u32917    1039
u4324      990
u45        976
          ... 
u23634       1
u23633       1
u23631       1
u23630       1
u9999        1
Name: UserID, Length: 45289, dtype: int64

In [ ]:
#create another user 
Checkin_u2651 = Checkin[Checkin.UserID == 'u2651']
Checkin_u2651.drop(['UserID'], axis=1, inplace=True)
Checkin_u2651.rename(columns={"Date":"Day"}, inplace=True)
Checkin_u2651.head()

,VenueID,Year,Month,Day,Hour
890,v68771,2012,2,24,11
2496,v68771,2012,2,25,17
2520,v68812,2012,2,25,19
3626,v68771,2012,2,25,11
4445,v69912,2012,2,25,13


In [ ]:
%%time
Checkin_u2651['Datetime'] = pd.to_datetime(Checkin_u2651[['Year', 'Month', 'Day', 'Hour']])
Checkin_u2651['Hour_mins'] = Checkin_u2651.Hour.values*60

CPU times: user 16.9 ms, sys: 4.02 ms, total: 21 ms
Wall time: 46.2 ms


In [ ]:
Checkin_u2651.head()

,VenueID,Year,Month,Day,Hour,Datetime,Hour_mins
890,v68771,2012,2,24,11,2012-02-24 11:00:00,660
2496,v68771,2012,2,25,17,2012-02-25 17:00:00,1020
2520,v68812,2012,2,25,19,2012-02-25 19:00:00,1140
3626,v68771,2012,2,25,11,2012-02-25 11:00:00,660
4445,v69912,2012,2,25,13,2012-02-25 13:00:00,780


In [ ]:
Checkin_u2651_nodup = Checkin_u2651.drop_duplicates('Datetime')
Checkin_u2651_nodup.sort_values(by='Datetime',inplace=True)
Checkin_u2651_nodup.iloc[:10]

,VenueID,Year,Month,Day,Hour,Datetime,Hour_mins
890,v68771,2012,2,24,11,2012-02-24 11:00:00,660
3626,v68771,2012,2,25,11,2012-02-25 11:00:00,660
4445,v69912,2012,2,25,13,2012-02-25 13:00:00,780
2496,v68771,2012,2,25,17,2012-02-25 17:00:00,1020
2520,v68812,2012,2,25,19,2012-02-25 19:00:00,1140
18581,v74594,2012,2,26,9,2012-02-26 09:00:00,540
19234,v69912,2012,2,26,11,2012-02-26 11:00:00,660
11835,v68771,2012,2,26,13,2012-02-26 13:00:00,780
15523,v61065,2012,2,26,15,2012-02-26 15:00:00,900
12367,v68775,2012,2,26,19,2012-02-26 19:00:00,1140


In [ ]:

%%time

import datetime

def _generate_events(data):
  previous_time = datetime.datetime(2020, 12, 31)
  all_events = []
  current_events = []
  for index, row in data.iterrows():
    current_time = row['Datetime']  
    current_hour = row['Hour']
    venue = row['VenueID']
    if( abs((current_time - previous_time).total_seconds())/60/60 < 8):
      current_events.append([venue, current_hour])
      previous_time = current_time
    else:
      all_events.append(current_events)
      current_events = []
      current_events.append([venue, current_hour])
      previous_time = current_time
  if len(current_events)>0:
    all_events.append(current_events)
  return all_events

CPU times: user 0 ns, sys: 7 µs, total: 7 µs
Wall time: 9.78 µs


In [ ]:
%%time
def _generate_subsequence(data):
  all_sequences = []
  for sequence in data:  
    if len(sequence) >= 2:
      sequences = []
      for i in range(0,len(sequence)-1):
          sequences.append(sequence[i:i+2])          
      all_sequences.append(sequences)
  return all_sequences

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.63 µs


In [ ]:
%time
o_events = _generate_events(Checkin_u2651_nodup)
print("Events")
print(o_events)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.68 µs
Events
[[], [['v68771', 11]], [['v68771', 11], ['v69912', 13], ['v68771', 17], ['v68812', 19]], [['v74594', 9], ['v69912', 11], ['v68771', 13], ['v61065', 15], ['v68775', 19]], [['v23056', 9], ['v68812', 16], ['v74594', 17], ['v68775', 19], ['v68771', 22]], [['v68771', 8], ['v69912', 11], ['v68768', 17], ['v68771', 18], ['v83111', 19], ['v68771', 21]], [['v68771', 6], ['v69912', 8], ['v64246', 9], ['v29805', 10], ['v18503', 12], ['v25312', 13], ['v27753', 16], ['v74594', 22], ['v68771', 23], ['v68775', 0], ['v68775', 7], ['v68812', 11], ['v18503', 12], ['v24143', 13], ['v82320', 19], ['v74594', 21], ['v9901', 23]], [['v68812', 9], ['v33365', 10]], [['v18503', 21], ['v69912', 22], ['v68771', 23]], [['v68771', 8], ['v68812', 9], ['v18503', 12], ['v7256', 17], ['v2641', 18]], [['v57948', 9], ['v29805', 10], ['v18503', 12], ['v1195', 14]], [['v68812', 22], ['v68775', 23], ['v69912', 0]], [['v68768', 8], ['v68812', 9], ['v44187

In [ ]:
def _set_earliest_event_time(data):
  all_sequences = []
  changed_data = data.copy()
  for sequence in changed_data:      
    if(len(sequence)>=2):
      sequence[0][1] = 0
      all_sequences.append(sequence)
  return all_sequences
    
m_events = _set_earliest_event_time(o_events)
print("Changed Events")
print(m_events)

Changed Events
[[['v68771', 0], ['v69912', 13], ['v68771', 17], ['v68812', 19]], [['v74594', 0], ['v69912', 11], ['v68771', 13], ['v61065', 15], ['v68775', 19]], [['v23056', 0], ['v68812', 16], ['v74594', 17], ['v68775', 19], ['v68771', 22]], [['v68771', 0], ['v69912', 11], ['v68768', 17], ['v68771', 18], ['v83111', 19], ['v68771', 21]], [['v68771', 0], ['v69912', 8], ['v64246', 9], ['v29805', 10], ['v18503', 12], ['v25312', 13], ['v27753', 16], ['v74594', 22], ['v68771', 23], ['v68775', 0], ['v68775', 7], ['v68812', 11], ['v18503', 12], ['v24143', 13], ['v82320', 19], ['v74594', 21], ['v9901', 23]], [['v68812', 0], ['v33365', 10]], [['v18503', 0], ['v69912', 22], ['v68771', 23]], [['v68771', 0], ['v68812', 9], ['v18503', 12], ['v7256', 17], ['v2641', 18]], [['v57948', 0], ['v29805', 10], ['v18503', 12], ['v1195', 14]], [['v68812', 0], ['v68775', 23], ['v69912', 0]], [['v68768', 0], ['v68812', 9], ['v44187', 14], ['v13815', 20], ['v2060', 22], ['v69912', 23], ['v68771', 0]], [['v68812'

In [ ]:
import random
n = len(m_events)  

n_test = int( n * .2 ) 
n_train = n - (n_test)

full_set = np.arange(n)
train_set = random.sample(range(n), n_train)
test_set = np.delete(full_set, train_set).tolist()

print(f"Dataset set terms: {n}")
print(f"Train set terms: {n_train}") 
print(f"Test set terms: {n_test}")


Dataset set terms: 119
Train set terms: 96
Test set terms: 23


In [ ]:
m_events = np.asarray(m_events)
train_data = m_events[train_set]  
test_data = m_events[test_set]

In [ ]:
%time
def _venue_only(data):
  all_sequences = []
  for sequence in data:
    events = []
    for event in sequence:      
      events.append(event[0])
    all_sequences.append(events)
  return all_sequences

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs


In [ ]:
%time
def _time_only(data):
  all_sequences = []
  for sequence in data:
    events = []
    for event in sequence:      
      events.append(event[1])
    all_sequences.append(events)
  return all_sequences


CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 6.91 µs


In [ ]:
#train data, split the features 

v_events_train = _venue_only(train_data)
print(v_events_train)
t_events_train = _time_only(train_data)
print(t_events_train)

[['v68812', 'v18503', 'v14225', 'v33028', 'v9775', 'v61065', 'v68775', 'v68812', 'v68768', 'v74594', 'v68775', 'v41716', 'v83111', 'v68768', 'v68812', 'v42978', 'v77584', 'v81623', 'v395', 'v18315', 'v575', 'v9901', 'v68775', 'v64246', 'v68812', 'v18503', 'v9498', 'v39147', 'v68775'], ['v68771', 'v68812', 'v18503'], ['v83111', 'v68771', 'v68812', 'v2060'], ['v68812', 'v68771'], ['v69912', 'v6127', 'v55121', 'v68812', 'v7382', 'v61065', 'v74594', 'v68775'], ['v55121', 'v68812', 'v18503', 'v47162', 'v13567', 'v6890', 'v61065'], ['v53888', 'v74594', 'v69912', 'v68775'], ['v18315', 'v9901', 'v68775', 'v68771', 'v68812', 'v18503', 'v8089', 'v1156', 'v20285', 'v3305', 'v39147', 'v9901', 'v69912', 'v68771', 'v57948', 'v83111', 'v18503', 'v28851'], ['v68812', 'v18503', 'v18610', 'v8738', 'v69912', 'v68775', 'v41716', 'v68812', 'v18503', 'v65878'], ['v68812', 'v68768'], ['v68812', 'v18503', 'v6890', 'v3305', 'v2060', 'v9901', 'v68812', 'v68771', 'v64246', 'v83111', 'v68768', 'v68812', 'v74594',

In [ ]:
#test data, split the features 

v_events_test = _venue_only(test_data)
print(v_events_test)
t_events_test = _time_only(test_data)
print(t_events_test)

[['v68771', 'v69912', 'v68771', 'v68812'], ['v68768', 'v68812', 'v44187', 'v13815', 'v2060', 'v69912', 'v68771'], ['v68771', 'v83111', 'v18503', 'v2133', 'v61065', 'v69912'], ['v68812', 'v11563', 'v4446', 'v74594', 'v68771', 'v69912'], ['v68775', 'v83111', 'v8056', 'v1195', 'v73617', 'v9901', 'v68771', 'v74594', 'v64246', 'v68812', 'v18503', 'v3897', 'v44187', 'v2641', 'v68812'], ['v18315', 'v9901'], ['v68771', 'v68812', 'v18503', 'v44187', 'v1156', 'v74594', 'v68771', 'v41716', 'v83111', 'v68812', 'v18503', 'v1195', 'v28851', 'v39147', 'v68775', 'v68771', 'v39147', 'v228', 'v230', 'v68812', 'v74594'], ['v41716', 'v83111', 'v68812', 'v18503', 'v9257', 'v1303', 'v19502', 'v44187', 'v38600', 'v74594', 'v68771', 'v68775', 'v57948', 'v64246', 'v83111', 'v68812', 'v18503', 'v6890', 'v3305', 'v18315', 'v9901', 'v68771', 'v68775', 'v41716', 'v68812', 'v11563', 'v18315', 'v68812'], ['v41716', 'v83111', 'v18503', 'v3305', 'v18315', 'v9901', 'v68771', 'v41716', 'v83111', 'v18503'], ['v2721', 'v7

In [ ]:
def _get_maxlength_seq(data):
  t = []
  for i in data:
    t.append([i,len(i)])
  temp_df = pd.DataFrame(t)
  temp_df.sort_values(by=1, ascending=False,inplace=True)
  return np.max(temp_df[1])

max_length_train= _get_maxlength_seq(train_data)
max_length_test= _get_maxlength_seq(test_data)

print(f"max_length_train: {max_length_train}")
print(f"max_length_test: {max_length_test}")

max_length_train: 38
max_length_test: 28


In [ ]:
def _all_venue_index():
  count = 1
  for venue in Checkin_u2651_nodup.VenueID.unique():
    all_venues[venue] = count
    count+=1
  return all_venues

all_venues = {}
all_venues = _all_venue_index()
all_venues_list = Checkin_u2651_nodup.VenueID.unique()
no_of_samples_train = len(train_data)
no_of_samples_test = len(test_data)
no_of_venues = len(Checkin_u2651_nodup.VenueID.unique())


In [ ]:
def _new_split(data):
  features = []
  targets = []
  for sequences in data:
    if(len(sequences)) >= 2:
      features.append(sequences[:-1])
      targets.append(sequences[-1])
  return features, targets


In [ ]:
features_s, targets_s = _new_split(v_events_train)
features_t, targets_t = _new_split(t_events_train)

In [ ]:
features_s_test, targets_s_test = _new_split(v_events_test)
features_t_test, targets_t_test = _new_split(t_events_test)

In [ ]:
print(features_s[1], targets_s[1])

print(features_t[1], targets_t[1])

['v68771', 'v68812'] v18503
[0, 12] 14


In [ ]:
#train x and y

x_train = np.zeros((no_of_samples_train, max_length_train, no_of_venues+1), dtype=np.bool)
y_train = np.zeros((no_of_samples_train, no_of_venues+1), dtype=np.bool)

def _one_hot(x,y,features_s,targets_s):
  for i, sequence in enumerate(features_s):
    for t, venue in enumerate(sequence):
        x[i, t, all_venues[venue]] = 1
    y[i, all_venues[targets_s[i]]] = 1

_one_hot(x_train, y_train, features_s,targets_s)

In [ ]:

x_test = np.zeros((no_of_samples_test, max_length_test, no_of_venues+1), dtype=np.bool)
y_test = np.zeros((no_of_samples_test, no_of_venues+1), dtype=np.bool)

def _one_hot(x,y,features_s_test,targets_s_test):
  for i, sequence in enumerate(features_s_test):
    for t, venue in enumerate(sequence):
        x_test[i, t, all_venues[venue]] = 1
    y_test[i, all_venues[targets_s_test[i]]] = 1

_one_hot(x_test, y_test, features_s_test,targets_s_test)

1. **LSTM**

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

Using TensorFlow backend.


In [ ]:
from keras.layers import Input, Dense, Flatten, Embedding
from keras.models import Model

In [ ]:
EPOCHS = 50
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 139   # number of outputs = number of digits
N_HIDDEN = 128
VALIDATION_SPLIT = 0.2 # how much TRAIN is reserved for VALIDATION

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(128, name='lstm_layer', input_shape=(max_length_train,no_of_venues+1)))
model.add(tf.keras.layers.Dense(no_of_venues+1, name='dense_layer', activation='softmax'))

model.summary()

#https://keras.io/examples/lstm_text_generation/

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_layer (LSTM)            (None, 128)               137728    
_________________________________________________________________
dense_layer (Dense)          (None, 140)               18060     
Total params: 155,788
Trainable params: 155,788
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
print('Train...')
model.fit(x_train, 
          y_train,
          epochs=1000,
          verbose=2)

Train...
Epoch 1/1000
3/3 - 0s - loss: 4.9382 - accuracy: 0.1458
Epoch 2/1000
3/3 - 0s - loss: 4.9208 - accuracy: 0.2188
Epoch 3/1000
3/3 - 0s - loss: 4.8771 - accuracy: 0.2188
Epoch 4/1000
3/3 - 0s - loss: 4.6621 - accuracy: 0.2188
Epoch 5/1000
3/3 - 0s - loss: 3.7237 - accuracy: 0.2188
Epoch 6/1000
3/3 - 0s - loss: 3.2685 - accuracy: 0.2188
Epoch 7/1000
3/3 - 0s - loss: 3.1357 - accuracy: 0.2188
Epoch 8/1000
3/3 - 0s - loss: 3.0301 - accuracy: 0.2188
Epoch 9/1000
3/3 - 0s - loss: 2.9817 - accuracy: 0.2188
Epoch 10/1000
3/3 - 0s - loss: 2.9340 - accuracy: 0.2188
Epoch 11/1000
3/3 - 0s - loss: 2.9029 - accuracy: 0.2188
Epoch 12/1000
3/3 - 0s - loss: 2.8873 - accuracy: 0.2188
Epoch 13/1000
3/3 - 0s - loss: 2.8641 - accuracy: 0.2188
Epoch 14/1000
3/3 - 0s - loss: 2.8564 - accuracy: 0.2188
Epoch 15/1000
3/3 - 0s - loss: 2.8463 - accuracy: 0.2188
Epoch 16/1000
3/3 - 0s - loss: 2.8399 - accuracy: 0.2188
Epoch 17/1000
3/3 - 0s - loss: 2.8375 - accuracy: 0.2188
Epoch 18/1000
3/3 - 0s - loss: 

In [ ]:
#@title
def _code_features(data):
  all_sequences = []
  for sequence in data:
    all_events = []
    for venue in sequence:
      venue = all_venues.get(venue)
      all_events.append(venue)
    all_sequences.append(all_events)
  return all_sequences

In [ ]:
#@title
def _target_code(targets_s):
  targets_s_codeded = np.zeros((no_of_samples_train,no_of_venues),dtype=np.bool)
  for index,data in enumerate(targets_s_codeded):
    venue_index = all_venues.get(targets_s[index])
    targets_s_codeded[index][venue_index] =1 
  return targets_s_codeded


2. **GRU Model**

In [ ]:
# Load libraries
import numpy as np
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras import models
from keras import layers

# Set random seed
np.random.seed(0)

In [ ]:
# Start neural network
network = models.Sequential()

network.add(layers.GRU(units=256,name='GRU_layer', input_shape=(max_length_train,no_of_venues+1)))
network.add(Dropout(0.25))
network.add(layers.Dense(units=no_of_venues+1, activation='softmax'))
network.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
GRU_layer (GRU)              (None, 256)               304896    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 140)               35980     
Total params: 340,876
Trainable params: 340,876
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile neural network
network.compile(loss='categorical_crossentropy', # Cross-entropy
                optimizer='Adam', # Adam optimization
                metrics=['accuracy']) # Accuracy performance metric

In [ ]:
# Train neural network
history = network.fit(x_train, # Features
                      y_train, # Target
                      epochs=1000, 
                      verbose=1)

Epoch 1/10000
96/96 [==============================] - 1s 10ms/step - loss: 4.9380 - accuracy: 0.1042
Epoch 2/10000
96/96 [==============================] - 1s 6ms/step - loss: 4.9179 - accuracy: 0.2292
Epoch 3/10000
96/96 [==============================] - 1s 6ms/step - loss: 4.8762 - accuracy: 0.2292
Epoch 4/10000
96/96 [==============================] - 1s 6ms/step - loss: 4.7434 - accuracy: 0.2292
Epoch 5/10000
96/96 [==============================] - 1s 6ms/step - loss: 4.1174 - accuracy: 0.2188
Epoch 6/10000
96/96 [==============================] - 1s 6ms/step - loss: 3.4107 - accuracy: 0.2188
Epoch 7/10000
96/96 [==============================] - 1s 6ms/step - loss: 3.3531 - accuracy: 0.2188
Epoch 8/10000
96/96 [==============================] - 1s 6ms/step - loss: 3.0546 - accuracy: 0.1979
Epoch 9/10000
96/96 [==============================] - 1s 6ms/step - loss: 2.9269 - accuracy: 0.2188
Epoch 10/10000
96/96 [==============================] - 1s 6ms/step - loss: 2.9458 - accur

KeyboardInterrupt: ignored

3. **Venue and Time**

https://datascience.stackexchange.com/questions/53609/how-to-determine-input-shape-in-keras

https://mmuratarat.github.io/2019-06-12/embeddings-with-numeric-variables-Keras#:~:text=In%20order%20to%20combine%20the,do%20for%20any%20regular%20network.

https://www.tensorflow.org/guide/keras/masking_and_padding

https://towardsdatascience.com/text-classifier-with-multiple-outputs-and-multiple-losses-in-keras-4b7a527eb858

https://keras.io/api/losses/probabilistic_losses/#categoricalcrossentropy-class

https://www.kaggle.com/rajmehra03/a-detailed-explanation-of-keras-embedding-layer

https://medium.com/@satnalikamayank12/on-learning-embeddings-for-categorical-data-using-keras-165ff2773fc9

https://keras.io/guides/working_with_rnns/

https://keras.io/guides/making_new_layers_and_models_via_subclassing/

https://towardsdatascience.com/neural-network-embeddings-explained-4d028e6f0526

https://stats.stackexchange.com/questions/270546/how-does-keras-embedding-layer-work

https://stackabuse.com/python-for-nlp-word-embeddings-for-deep-learning-in-keras/